<a href="https://colab.research.google.com/github/awaris123/OSNA-Project2/blob/main/GPUcolab_train_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -U sentence-transformers
from sentence_transformers import SentenceTransformer


     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.9-cp36-none-any.whl size=101036 sha256=a7c1c357697693031192c2c71b614d5edabe0e20400081025179f95ce589747c
  Stored in directory: /root/.cache/pip/wheels/fc/89/43/f2f5bc00b03ef9724b0f6254a97eaf159a4c4ddc024b33e07a
Successfully built sentence-transformers


In [ ]:
import numpy as np
import keras
import pandas as pd
import pdb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [ ]:
#    def __init__(self, df, y_data, dim=(768*2), batch_size=200, n_classes=3, shuffle=True):
#from simple_classifier_model import SimpleClassifier
!pip3 install transformers
from xlnet import XLNet

df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
label_encoder = LabelEncoder()
targets = label_encoder.fit_transform(df['label'].values)

X_train, X_test, y_train, y_test = train_test_split(df[['title1_en', 'title2_en']], targets, test_size=0.2)
df_train = pd.DataFrame()

params={'lr': 0.01, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 25000}



In [ ]:
#for xlnet
from data_generator import DataGenerator
import pickle

training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])
xlnet1= XLNet()
model = xlnet1.model
model.summary()
def get_inputs(tweets, tokenizer, max_len=120):
    """ Gets tensors from text using the tokenizer provided"""
    inps = [tokenizer.encode_plus(t, max_length=max_len, pad_to_max_length=True, add_special_tokens=True) for t in tweets]
    inp_tok = np.array([a['input_ids'] for a in inps])
    ids = np.array([a['attention_mask'] for a in inps])
    segments = np.array([a['token_type_ids'] for a in inps])
    return inp_tok, ids, segments

inp_tok, ids, segments = get_inputs(df[['title1_en', 'title2_en']], xlnet_tokenizer)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=params['lr'],
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=params['lr'])

model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])#, keras.metrics.Precision(), keras.metrics.Recall()])

'''
model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=10,
                          workers=8,
                    use_multiprocessing=False,
                    verbose=1)
'''
model.save("/content/drive/MyDrive/joint_embeddings/test/xlnet_ft_unbalanced")

Some layers from the model checkpoint at xlnet-large-cased were not used when initializing TFXLNetModel: ['lm_loss']
- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLNetModel were initialized from the model checkpoint at xlnet-large-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLNetModel for predictions without further training.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting

Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_inputs (InputLayer)     [(None, 128)]             0         
_________________________________________________________________
tfxl_net_model_7 (TFXLNetMod ((None, 128, 1024),)      360268800 
_________________________________________________________________
tf_op_layer_strided_slice_7  [(None, 1, 1024)]         0         
_________________________________________________________________
tf_op_layer_Squeeze_7 (Tenso [(None, 1024)]            0         
_________________________________________________________________
dropout_591 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 3075      
Total params: 360,271,875
Trainable params: 360,271,875
Non-trainable params: 0
_______________________________________

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


AttributeError: ignored

In [ ]:
from data_generator import DataGenerator
import pickle

training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])
all_generator = DataGenerator(df, targets, total_samples=targets.shape[0], batch_size=params['batch_size'])
joint_embedding = np.empty((all_generator.total_samples, *all_generator.dim))
y = np.empty((all_generator.total_samples), dtype=int)

for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  e1, y1=all_generator.__getitem__(i) #

  with open(f"./joint_embedding_{i}", 'wb') as f:
    pickle.dump(e1, f)

  with open(f"./y_{i}", 'wb') as f:
    pickle.dump(y1, f)

0 / 256442
25000 / 256442
50000 / 256442
75000 / 256442
100000 / 256442
125000 / 256442
150000 / 256442
175000 / 256442
200000 / 256442
225000 / 256442


In [ ]:
del df
del targets
import pickle
with open("./joint_embedding", 'wb') as f:
  pickle.dump(joint_embedding, f)

with open("./y", 'wb') as f:
  pickle.dump(y, f)

NameError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mkdir ./drive/MyDrive/joint_embeddings


In [ ]:
!cp -r joint_embedding ./drive/MyDrive/joint_embeddings/
!cp -r y ./drive/MyDrive/joint_embeddings/

cp: cannot stat 'joint_embedding': No such file or directory
cp: cannot stat 'y': No such file or directory


In [ ]:
import pickle
path="./drive/MyDrive/joint_embeddings/"
batch_size=25000
X_all = np.empty((batch_size*10, 1536))
y_all = np.empty((batch_size*10))


for i in range(10):
  with open(f"{path}/joint_embedding_{i}", 'rb') as f:
    X=pickle.load(f)
    X_all[i*batch_size:(i+1)*batch_size,:]=X

  with open(f"{path}/y_{i}", 'rb') as f:
    y=pickle.load(f)
    y_all[i*batch_size:(i+1)*batch_size]=y
  print(i)



0
1
2
3
4
5


KeyboardInterrupt: ignored

In [ ]:
with open("./joint_embedding", 'wb') as f:
  pickle.dump(X_all, f)

with open("./y", 'wb') as f:
  pickle.dump(y_all, f)

KeyboardInterrupt: ignored

In [ ]:
#load
import pickle
path="./drive/MyDrive/joint_embeddings/"

with open(f"{path}joint_embedding", 'rb') as f:
  X_all=pickle.load(f)

with open(f"{path}y", 'rb') as f:
  y_all=pickle.load(f)

In [ ]:
!cp /content/drive/MyDrive/joint_embeddings/data_generator.py .
!cp /content/drive/MyDrive/joint_embeddings/simple_classifier_model.py .

In [ ]:
#train from preprocessed data
from data_generator import DataGenerator
import pickle

params={'lr': 0.001, 'opt': 'adam', 'model': 'SimpleClassifier', 'loss': 'crossentropy', 'batch_size': 128}

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2)
del X_all
del y_all
training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=params['batch_size'])
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])

In [ ]:
import simple_classifier_model #import SimpleClassifier
import importlib
importlib.reload(simple_classifier_model)

model = simple_classifier_model.SimpleClassifier(input_shape=768*2)
model.model.summary()

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=params['lr'],
    decay_steps=10000,
    decay_rate=0.9)
opt = keras.optimizers.Adam(learning_rate=params['lr'])

model.model.compile(optimizer=opt, loss=keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])#, keras.metrics.Precision(), keras.metrics.Recall()])
model.model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=100,
                          workers=8,
                    use_multiprocessing=False,
                    verbose=1)
model.model.save("/content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500_512_bs128_leakyrelu")

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 1500)              2305500   
_________________________________________________________________
batch_normalization_9 (Batch (None, 1500)              6000      
_________________________________________________________________
dropout_9 (Dropout)          (None, 1500)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 250)               375250    
_________________________________________________________________
batch_normalization_10 (Batc (None, 250)               1000      
_________________________________________________________________
dropout_10 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 50)               

INFO:tensorflow:Assets written to: /content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500_512_bs128_leakyrelu/assets


In [ ]:
# EVALUATION
import data_generator #import DataGenerator
importlib.reload(data_generator)

#model=keras.models.load("/content/drive/MyDrive/joint_embeddings/test/bert_mlp_unbalanced_1500")
df=pd.read_csv("https://raw.githubusercontent.com/awaris123/OSNA-Project2/main/option1-data/train.csv")
encoder = LabelEncoder()
encoder.fit_transform(df['label'].values)

df=pd.read_csv("/content/drive/MyDrive/joint_embeddings/test.csv", sep=',')
final_df=pd.DataFrame(data=None, columns=['id', 'label'])
final_df['id'] = df['id']
targets=np.empty((final_df.shape[0]), dtype=int)
all_generator = data_generator.DataGenerator(df, targets, n_classes=3, total_samples=targets.shape[0], batch_size=128, pretrained_embeddings=False)
print(int(all_generator.total_samples/all_generator.batch_size))
for i in range(int(all_generator.total_samples/all_generator.batch_size)):
  #joint_embedding[i*all_generator.batch_size:(i+1)*all_generator.batch_size,:], y[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=all_generator.__getitem__(i) #
  print(i, end = ' ')
  e1, _=all_generator.__getitem__(i) #
  targets[i*all_generator.batch_size:(i+1)*all_generator.batch_size]=np.argmax(model.model.predict(e1), axis=1)
  print(i, end = ' ')


#label decoder
#final_df['label'] = encoder.inverse_transform(targets)

#convert to csv
#final_df.to_csv("/content/drive/MyDrive/joint_embeddings/sample_submission.csv", index=False)

500
0 0 1 1 2 2 3 3 4 4 5 5 6 6 7 7 8 8 9 9 10 10 11 11 12 12 13 13 14 14 15 15 16 16 17 17 18 18 19 19 20 20 21 21 22 22 23 23 24 24 25 25 26 26 27 27 28 28 29 29 30 30 31 31 32 32 33 33 34 34 35 35 36 36 37 37 38 38 39 39 40 40 41 41 42 42 43 43 44 44 45 45 46 46 47 47 48 48 49 49 50 50 51 51 52 52 53 53 54 54 55 55 56 56 57 57 58 58 59 59 60 60 61 61 62 62 63 63 64 64 65 65 66 66 67 67 68 68 69 69 70 70 71 71 72 72 73 73 74 74 75 75 76 76 77 77 78 78 79 79 80 80 81 81 82 82 83 83 84 84 85 85 86 86 87 87 88 88 89 89 90 90 91 91 92 92 93 93 94 94 95 95 96 96 97 97 98 98 99 99 100 100 101 101 102 102 103 103 104 104 105 105 106 106 107 107 108 108 109 109 110 110 111 111 112 112 113 113 114 114 115 115 116 116 117 117 118 118 119 119 120 120 121 121 122 122 123 123 124 124 125 125 126 126 127 127 128 128 129 129 130 130 131 131 132 132 133 133 134 134 135 135 136 136 137 137 138 138 139 139 140 140 141 141 142 142 143 143 144 144 145 145 146 146 147 147 148 148 149 149 150 150 151 151 

In [ ]:
len([i for i in targets if i > 2])
le_name_mapping = dict(zip(encoder.transform(encoder.classes_), encoder.classes_))
le_name_mapping
final_df['label']=[le_name_mapping.get(i, 'unrelated') for i in targets]

#label decoder

#convert to csv
final_df.to_csv("/content/drive/MyDrive/joint_embeddings/sample_submission.csv", index=False)

In [ ]:
training_generator = DataGenerator(X_train, y_train, total_samples=y_train.shape[0], batch_size=64)
validation_generator = DataGenerator(X_test, y_test, total_samples=y_test.shape[0], batch_size=params['batch_size'])
model.model.fit_generator(generator=training_generator,
                          validation_data=validation_generator,
                          epochs=100,
                          workers=8,
                    use_multiprocessing=False,
                    verbose=1)

Epoch 1/100
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1538 - accuracy: 0.9401 - val_loss: 0.5010 - val_accuracy: 0.8488
Epoch 2/100
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1452 - accuracy: 0.9431 - val_loss: 0.4879 - val_accuracy: 0.8515
Epoch 3/100
3125/3125 [==============================] - 17s 6ms/step - loss: 0.1438 - accuracy: 0.9435 - val_loss: 0.4871 - val_accuracy: 0.8535
Epoch 4/100
3125/3125 [==============================] - 17s 6ms/step - loss: 0.1415 - accuracy: 0.9448 - val_loss: 0.4845 - val_accuracy: 0.8544
Epoch 5/100
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1412 - accuracy: 0.9445 - val_loss: 0.4968 - val_accuracy: 0.8523
Epoch 6/100
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1395 - accuracy: 0.9455 - val_loss: 0.4625 - val_accuracy: 0.8549
Epoch 7/100
3125/3125 [==============================] - 17s 5ms/step - loss: 0.1397 - accuracy: 0.9449 - val_loss: 0.4834 -